In [1]:
# Cell 1
import pandas as pd
import numpy as np
import nltk
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import tensorflow as tf

2023-10-31 20:51:01.227440: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Cell 2
# Load your Excel files for validation
validation_file_path = '/Users/PremGanesh/Developer/Cyvidia/CyVidia/Input_Data/Validation Dataset.xlsx'
validation_df = pd.read_excel(validation_file_path)
# Define input and output file paths
input_file_path = '/Users/PremGanesh/Developer/Cyvidia/CyVidia/Output_Data/mlv.xlsx'
output_file_path = '/Users/PremGanesh/Developer/Cyvidia/CyVidia/Output_Data/mlv_validation2.xlsx'

In [3]:
# Cell 3
# Load the saved tokenizer using pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [4]:
# Cell 4
# Load the encoders from the training process
with open('area_encoder.pickle', 'rb') as handle:
    area_encoder = pickle.load(handle)
with open('bucket_encoder.pickle', 'rb') as handle:
    bucket_encoder = pickle.load(handle)


In [5]:

# Cell 5
# Define a function for text cleaning
def clean_text(text):
    if isinstance(text, float) and np.isnan(text):
        return ""
    words = word_tokenize(text)
    words = [word.lower() for word in words if word.isalnum()]
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    clean_text = ' '.join(words)
    return clean_text


In [6]:
# Cell 6
# Apply text cleaning to 'Requirement Description' column for validation data
validation_df['Cleaned_Description'] = validation_df['Requirement Description'].apply(clean_text)



In [7]:
# Cell 7
# Tokenize and pad sequences for validation data
max_words = 1000
X_val = tokenizer.texts_to_sequences(validation_df['Cleaned_Description'])
X_val = pad_sequences(X_val, maxlen=100)

In [8]:
# Cell 8
# Load the saved model
loaded_model = load_model('trained_model')

In [9]:
# Cell 9
# Predict on validation data using the loaded model
y_pred = loaded_model.predict(X_val)

5/5 [==============================] - 1s 36ms/step


In [10]:
# Cell 10
# Get prediction scores for 'Area' and 'Bucket'
prediction_scores_area = np.max(y_pred[0], axis=1)
prediction_scores_bucket = np.max(y_pred[1], axis=1)

In [11]:
# Cell 11
# Add the prediction scores to the validation DataFrame
validation_df['Prediction_Score_Area'] = prediction_scores_area
validation_df['Prediction_Score_Bucket'] = prediction_scores_bucket


In [12]:
# Cell 12
# Define a labeling threshold (e.g., 0.85)
labeling_threshold = 0.85

In [13]:
# Cell 13
# Initialize lists to store predicted labels and suggested labels
predicted_labels_area = []
suggested_labels_area = []
predicted_labels_bucket = []
suggested_labels_bucket = []

In [14]:
# Cell 14
# Iterate through predictions and apply labeling/suggestion logic
for pred_area, score_area, pred_bucket, score_bucket in zip(
    np.argmax(y_pred[0], axis=1),
    prediction_scores_area,
    np.argmax(y_pred[1], axis=1),
    prediction_scores_bucket
):
    if score_area >= labeling_threshold:
        predicted_labels_area.append(area_encoder.inverse_transform([pred_area])[0])
        suggested_labels_area.append('')
    else:
        predicted_labels_area.append('Other')
        suggested_labels_area.append(area_encoder.inverse_transform([pred_area])[0])

    if score_bucket >= labeling_threshold:
        predicted_labels_bucket.append(bucket_encoder.inverse_transform([pred_bucket])[0])
        suggested_labels_bucket.append('')
    else:
        predicted_labels_bucket.append('Other')
        suggested_labels_bucket.append(bucket_encoder.inverse_transform([pred_bucket])[0])


In [15]:
# Cell 15
# Add the predicted labels and suggested labels to the validation DataFrame
validation_df['Predicted_Area'] = predicted_labels_area
validation_df['Suggested_Area'] = suggested_labels_area
validation_df['Predicted_Bucket'] = predicted_labels_bucket
validation_df['Suggested_Bucket'] = suggested_labels_bucket

In [16]:

# Cell 16
# Save the validation results DataFrame to an Excel file
validation_results_file_path = '/Users/PremGanesh/Developer/Cyvidia/CyVidia/Output_Data/mlv.xlsx'
validation_df.to_excel(validation_results_file_path, index=False)
print("Validation results with predicted/suggested labels and scores saved to", validation_results_file_path)

Validation results with predicted/suggested labels and scores saved to /Users/PremGanesh/Developer/Cyvidia/CyVidia/Output_Data/mlv.xlsx


In [17]:
# Define column names
req_no_col = 'ReqNo'
req_area_col = 'Requirement Area'
req_area_nist_col = 'Requirement Area (NIST)'
req_bucket_nist_col = 'Requirement Bucket(NIST)'

key_words_col = 'Key Words'
type_col = 'Type'
source_type_col = 'Source Type'
source_col = 'Source'
source_detail_col = 'Source Detail'
source_req_id_col = 'Source Requirement ID#'


In [18]:
# Read the Excel file
df = pd.read_excel(input_file_path)


In [19]:
# Convert the predicted_area into lower case
df['Requirement Area (NIST)'] = df['Requirement Area (NIST)'].str.lower()
df['Requirement Bucket(NIST)'] = df['Requirement Bucket(NIST)'].str.lower()


In [20]:
# Define a function to calculate validation
def calculate_validation(row):
    # if predicted area is other then check with suggested area with nist area
    if row['Predicted_Area'] == 'Other':
        if row['Suggested_Area'] == row[req_area_nist_col]:
            area_validation = 'Correct'
        elif pd.isna(row['Suggested_Area']):
            area_validation = 'Empty'
        else:
            area_validation = 'Incorrect'
    if pd.notna(row['Predicted_Area']) and row['Predicted_Area'] == row[req_area_nist_col]:
        area_validation = 'Correct'
    elif pd.isna(row['Predicted_Area']):
        area_validation = 'Empty'
    else:
        area_validation = 'Incorrect'

    if pd.notna(row['Predicted_Bucket']) and row['Predicted_Bucket'] == row[req_bucket_nist_col]:
        bucket_validation = 'Correct'
    elif pd.isna(row['Predicted_Bucket']):
        bucket_validation = 'Empty'
    else:
        bucket_validation = 'Incorrect'

    return area_validation, bucket_validation


In [21]:
# Apply the function to create new columns
df['Area Validation'], df['Bucket Validation'] = zip(*df.apply(calculate_validation, axis=1))


In [22]:
# Calculate counts for 'Area Validation' and 'Bucket Validation'
area_validation_counts = df['Area Validation'].value_counts()
bucket_validation_counts = df['Bucket Validation'].value_counts()


In [23]:
# Calculate percentages for 'Area Validation Accuracy' and 'Bucket Validation Accuracy'
area_validation_accuracy = (area_validation_counts.get('Correct', 0) / (area_validation_counts.sum())) * 100
bucket_validation_accuracy = (bucket_validation_counts.get('Correct', 0) / (bucket_validation_counts.sum())) * 100


In [24]:
# Create a dictionary with the counts and percentages
validation_summary = {
    'Total Count': len(df),
    'Area Validation Correct Count': area_validation_counts.get('Correct', 0),
    'Area Validation Incorrect Count': area_validation_counts.get('Incorrect', 0),
    'Area Validation Empty Count': area_validation_counts.get('Empty', 0),
    'Bucket Validation Correct Count': bucket_validation_counts.get('Correct', 0),
    'Bucket Validation Incorrect Count': bucket_validation_counts.get('Incorrect', 0),
    'Bucket Validation Empty Count': bucket_validation_counts.get('Empty', 0),
    'Area Validation Accuracy': area_validation_accuracy,
    'Bucket Validation Accuracy': bucket_validation_accuracy
}


In [ ]:
# Do analysis on the validation summary which are correctly mapped and which are wrongly mapped
df_correct = df[(df['Area Validation'] == 'Correct') & (df['Bucket Validation'] == 'Correct')]
df_incorrect = df[(df['Area Validation'] == 'Incorrect') | (df['Bucket Validation'] == 'Incorrect')]
df_empty = df[(df['Area Validation'] == 'Empty') | (df['Bucket Validation'] == 'Empty')]
# add plot for the validation summary



In [25]:
# Convert the dictionary to a DataFrame and transpose it
summary_df = pd.DataFrame(validation_summary, index=[0]).T


In [26]:
# Reset the index
summary_df = summary_df.reset_index()
# Rename the columns
summary_df.columns = ['Validation Metric', 'Count']
# Append the summary DataFrame to the original DataFrame
df = pd.concat([df, summary_df], ignore_index=True)
# Drop specified columns
columns_to_remove = [req_no_col, req_area_col, 'Requirements Bucket','Cleaned_Description' , key_words_col, type_col, source_type_col, source_col, source_detail_col, source_req_id_col]
df = df.drop(columns=columns_to_remove)
# Save the DataFrame to a new Excel file
df.to_excel(output_file_path, index=False)
